In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def iterate_data(data, batchsize, shuffle=False):
    if shuffle:
        indices = np.arange(data.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, data.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield data[excerpt, :, :]

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from torch.autograd import Variable
from torch.distributions import Categorical
import numpy as np
import math

def log_sum_exp(x, dim=1):
    x_max, x_argmax = x.max(dim, keepdim=True)
    x_max_broadcast = x_max.expand(*x.size())
    return x_max + torch.log(
        torch.sum(torch.exp(x - x_max_broadcast), dim=dim, keepdim=True))

class MDN(nn.Module):
    """A mixture density network layer
    The input maps to the parameters of a MoG probability distribution, where
    each Gaussian has O dimensions and diagonal covariance.
    Arguments:
        in_features (int): the number of dimensions in the input
        out_features (int): the number of dimensions in the output
        num_gaussians (int): the number of Gaussians per output dimensions
    Input:
        minibatch (BxD): B is the batch size and D is the number of input
            dimensions.
    Output:
        (pi, sigma, mu) (BxG, BxGxO, BxGxO): B is the batch size, G is the
            number of Gaussians, and O is the number of dimensions for each
            Gaussian. Pi is a multinomial distribution of the Gaussians. Sigma
            is the standard deviation of each Gaussian. Mu is the mean of each
            Gaussian.
    """

    def __init__(self, in_features, out_features, num_gaussians):
        super(MDN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.num_gaussians = num_gaussians
        self.pi = nn.Sequential(
            nn.Linear(in_features, num_gaussians), nn.LogSoftmax(dim=1))
        self.sigma = nn.Linear(in_features, out_features * num_gaussians)
        self.mu = nn.Linear(in_features, out_features * num_gaussians)

    def forward(self, minibatch):
        pi = self.pi(minibatch)
        sigma = self.sigma(minibatch)
        # original sigma = torch.clamp(sigma, np.log(np.sqrt(1e-4)), 1e8)
        # working 
        sigma =  torch.clamp(sigma, np.log(np.sqrt(1e-3)), 5e1)
        #try 3 sigma = torch.clamp(sigma, np.log(np.sqrt(1e-3)), 1e4)
        sigma = sigma.view(-1, self.num_gaussians, self.out_features)
        mu = self.mu(minibatch)
        mu = mu.view(-1, self.num_gaussians, self.out_features)
        return pi, sigma, mu

    @staticmethod
    def gaussian_probability(sigma, x_mu, x):
        """Returns the probability of `data` given MoG parameters `sigma` and `mu`.
        Arguments:
            sigma (BxGxO): The standard deviation of the Gaussians. B is the batch
                size, G is the number of Gaussians, and O is the number of
                dimensions per Gaussian.
            mu (BxGxO): The means of the Gaussians. B is the batch size, G is the
                number of Gaussians, and O is the number of dimensions per Gaussian.
            data (BxI): A batch of data. B is the batch size and I is the number of
                input dimensions.
        Returns:
            probabilities (BxG): The probability of each point in the probability
                of the distribution in the corresponding sigma/mu index.
        """
        x = x.unsqueeze(1).expand_as(sigma)
        var = (torch.exp(sigma)**2)
        return -((x - x_mu)**2) / (2 * var + 1e-4) - sigma - math.log(
            math.sqrt(2 * math.pi))

    @staticmethod
    def mdn_loss(pi, sigma, mu, target):
        """Calculates the error, given the MoG parameters and the target
        The loss is the negative log likelihood of the data given the MoG
        parameters.
        """
        nll = log_sum_exp(pi[:, :, None] +
                          MDN.gaussian_probability(sigma, mu, target))
        nll = -torch.sum(nll, dim=-1)
        return torch.mean(nll)

    @staticmethod
    def sample(pi, sigma, mu):
        """Draw samples from a MoG.
        """
        categorical = Categorical(torch.exp(pi))
        pis = list(categorical.sample().data)
        sigma = torch.exp(sigma)
        sample = Variable(
            sigma.data.new(sigma.size(0), sigma.size(2)).normal_())
        for i, idx in enumerate(pis):
            sample[i] = sample[i].mul(sigma[i, idx]).add(mu[i, idx])
        return sample

In [ ]:
def initialize_weights(model):
    if type(model) in [nn.Linear]:
        nn.init.xavier_normal_(model.weight.data)
    elif type(model) in [nn.LSTM, nn.RNN, nn.GRU]:
        nn.init.xavier_normal_(model.weight_hh_l0)
        nn.init.xavier_normal_(model.weight_ih_l0)

In [ ]:
import torch.autograd as autograd

class SimpleRNN(torch.nn.Module):
    def __init__(self, n_features, n_outputs):
        super(SimpleRNN, self).__init__()
        # 32 was used for all the simulated data
        #hidden_dim = 32 #
        
        hidden_dim = 128 #hidden_dim

        #self.inp = torch.nn.Linear(n_features, hidden_size)
        num_layers = 2
        #self.rnn = LayerNormLSTM(n_features, hidden_dim, num_layers = num_layers)
        self.rnn = torch.nn.LSTM(n_features, hidden_dim, num_layers = num_layers)
        
        # 64 was used for all the simulated data
        #self.out = torch.nn.Linear(hidden_dim, 64)
        
        #self.out = torch.nn.Linear(hidden_dim, 32)
        self.mdn = MDN(hidden_dim, n_outputs, 5)

        
        #self.hidden = None
        
        initialize_weights(self.rnn)
        #initialize_weights(self.out)
        initialize_weights(self.mdn)

    def step(self, inputs, hidden=None, verbose=False):
        #input = self.inp(input)
        if verbose:
            print("Step 0:")
            print(inputs.shape)
        inputs = inputs.permute([1, 0, 2])
        if verbose:
            print("Step 1:")
            print(inputs.shape)
        #self.rnn.flatten_parameters()
        output, hidden = self.rnn(inputs, hidden)
        output = output[-1, :, :] #output[:, :, :] #output[-1, :, :]
        #output = output.permute([1, 0, 2])
        if verbose:
            print("Step 3:")
            print(output.shape)
        output = output.squeeze()
        if verbose:
            print("Step 4:")
            print(output.shape)
        #output = self.out(output)
        if verbose:
            print("Step 5:")
            print(output.shape)
            print(output)
        output = self.mdn(output)
        return output, hidden

    def forward(self, inputs, hidden=None, verbose=False):
        if verbose:
            print("inputs size: ", inputs.size)
        batch_size = inputs.size(0)    
        output, hidden = self.step(inputs, hidden, verbose=verbose)
        return output, hidden

In [ ]:
vision_bins = 16
#rnn = SimpleRNN(n_features=5 + (vision_bins * 8), n_outputs=4).cuda()
rnn = SimpleRNN(n_features=5 +(vision_bins * 8), n_outputs=4).cuda()
#rnn = torch.load('asd.pt')
rnn = torch.load('couzinmodel-rnn.pt')
rnn.eval()

# T-SNE Simulation

In [ ]:
import tables
hdf5_file = tables.open_file('pathtofile.h5', mode='r')
train = hdf5_file.root.train
print(train.shape)
labels = []
for i in range(0, 5100):
    batch_data = train[i, :, :]
    print(batch_data[-1, 0:4])
    #labels.append(np.where(np.array([1, 0, 1, 0]) == 1)) # np.where(batch_data[:, 0:4] == 1))
    for l in  np.where(batch_data[-1, 0:4] == 1)[0]:
        labels.append(l)
        print(l)

labels = np.asarray(labels)
print("lshape", labels.shape)

unique, counts = np.unique(labels, return_counts=True)
# array = np.array([self.getX(), self.getY(), self.orientation, 0.333, 0, 0, 
#                          0, dx, d<y, np.cos(self.orientation), np.sin(self.orientation)])
#        self.history.append(np.concatenate([np.concatenate([self.label, array]), self.vision]))#

print("labels: ", unique)
print("counts: ", counts)
hdf5_file.close()

In [ ]:
plt.hist(labels)

In [ ]:
for l in np.where(np.array([0, 0, 1, 0]) == 1)[0]:
    print(l)

In [ ]:
import tables 
import numpy as np
import torch
from IPython.display import clear_output

hdf5_file = tables.open_file('pathtofile.h5', mode='r')
# features; 0: age; 1 bee_id; rest: normal data
train = hdf5_file.root.train
print(train.shape)
hidds = []
labels = []
labels_importance = []
labels_wall = []
labels_zoa = []
labels_zoo = []
labels_zor = []
labels_normal = []
printcounter = 0
for i in range(0, 100000):
    if (printcounter == 1000):
        clear_output()
        print('Progress report: ', i/100000)
        printcounter = 0
    printcounter += 1
    batch_data = train[i, :, :]
    label = batch_data[-1, :4]
    #print(label)
    #print(label[3])
    p = 1 
    if p >= np.random.uniform(0.0, 1.0):
        batch_X = batch_data[:-1, 10:].astype(np.float32)[None, :, :]
        batch_X = np.insert(batch_X,[1],batch_X[0],axis=0)
        batch_X = torch.from_numpy(batch_X)
        batch_X = torch.autograd.Variable(batch_X).cuda()
        Y_predicted, hidden = rnn.forward(batch_X, verbose=False)
        hidds.append(np.concatenate((hidden[0].data.cpu().numpy()[1, 0, :],hidden[1].data.cpu().numpy()[1, 0, :])))
        if np.where(label == 1)[0].shape[0] > 1:
            labels.append(np.where(label == 1)[0][1] + 1)
        elif np.where(label == 1)[0].shape[0] > 0:
            labels.append(np.where(label == 1)[0][0] + 1)
        else:
            labels.append(0)
        #print(label)
        if label[3] > 0:
            labels_importance.append(3) #avoid wall
        elif label[2] > 0:
            labels_importance.append(2) #attraction zone filled
        elif label[1] > 0:
            labels_importance.append(1) #orientation zone
        elif label[0]> 0:
            labels_importance.append(0) #repulsion zone
        else:
            labels_importance.append(4) # no label -> normal walk
            
        #------------------------------------------------
        labels_wall.append(label[3])
        labels_zoa.append(label[2])
        labels_zoo.append(label[1])
        labels_zor.append(label[0])
        if np.array_equal(label, np.array([0, 0, 0, 0])):
            labels_normal.append(1)
        else:
            labels_normal.append(0)
hdf5_file.close()

In [ ]:
len(hidds)

In [ ]:
print("labels_wall: ", np.sum(labels_wall))
print("labels_zoa: ", np.sum(labels_zoa))
print("labels_zoo: ", np.sum(labels_zoo))
print("labels_zor: ", np.sum(labels_zor))
print("labels_normal: ", np.sum(labels_normal))

In [ ]:
plt.hist(labels_importance)

In [ ]:
plt.hist(labels_wall)

In [ ]:
plt.hist(labels_zoa)

In [ ]:
plt.hist(labels_zoo)

In [ ]:
plt.hist(labels_zor)

In [ ]:
x = np.arange(4)
money = np.histogram(labels_importance, bins= [0, 1, 2, 3, 4])[0]


def millions(x, pos):
    'The two args are the value and tick position'
    return '$%1.1fM' % (x * 1e-6)


#formatter = FuncFormatter(millions)

fig, ax = plt.subplots()
#ax.yaxis.set_major_formatter(formatter)
plt.bar(x, money)
plt.xticks(x, ('zor', 'zoo', 'zoa', 'avoid wall'))
plt.show()

In [ ]:
hidds = np.asarray(hidds)
labels = np.asarray(labels)
labels_importance = np.asarray(labels_importance)
labels_wall = np.asarray(labels_wall)
labels_zoa = np.asarray(labels_zoa)
labels_zoo = np.asarray(labels_zoo)
labels_zor = np.asarray(labels_zor)
labels_normal = np.asarray(labels_normal)
print(hidds.shape)
print(labels.shape)

In [ ]:
#from sklearn.manifold import TSNE
#https://github.com/DmitryUlyanov/Multicore-TSNE}
from MulticoreTSNE import MulticoreTSNE as TSNE

X_embedded = TSNE(n_components=2, perplexity = 50).fit_transform(hidds)
X_embedded.shape

In [ ]:
from matplotlib.ticker import NullFormatter

perplexities = [0, 5, 30, 50, 100]
(fig, subplots) = plt.subplots(3, 2, figsize=(11, 13))

fig.subplots_adjust(wspace=0.02, hspace=0.02)

ax = subplots[0, 0]
#ax.set_title("Perplexity=%d" % perplexity)
colors = ['whitesmoke', 'black']
ax.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_normal), cmap=matplotlib.colors.ListedColormap(colors), s=0.01)
black_patch = mpatches.Patch(color='black', label='normal walk')
ax.legend(handles=[black_patch])
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis('tight')

ax = subplots[0, 1]
colors = ['whitesmoke', 'gray']
ax.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_wall), cmap=matplotlib.colors.ListedColormap(colors), s=0.01)
gray_patch = mpatches.Patch(color='gray', label='avoid wall')
ax.legend(handles=[gray_patch])
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis('tight')


ax = subplots[1, 0]
colors = ['whitesmoke', 'red']
ax.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_zor), cmap=matplotlib.colors.ListedColormap(colors), s=0.01)
red_patch = mpatches.Patch(color='red', label='zor')
ax.legend(handles=[red_patch])
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis('tight')



ax = subplots[1, 1]
#ax.set_title("Perplexity=%d" % perplexity)
colors = ['whitesmoke', 'yellow']
ax.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_zoo), cmap=matplotlib.colors.ListedColormap(colors), s=0.01)
yellow_patch = mpatches.Patch(color='yellow', label='zoo')
ax.legend(handles=[yellow_patch])
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis('tight')


ax = subplots[2, 0]
#ax.set_title("Perplexity=%d" % perplexity)
colors = ['whitesmoke', 'blue']
ax.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_zoa), cmap=matplotlib.colors.ListedColormap(colors), s=0.01)
blue_patch = mpatches.Patch(color='blue', label='zoa')
ax.legend(handles=[blue_patch])
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis('tight')


#TSNE single labels only


ax = subplots[2, 1]
#ax.set_title("Perplexity=%d" % perplexity)
colors = ['black', 'red', 'yellow', 'blue', 'gray', 'orange', 'pink', 'darkred', 'green', 'khaki', 'darkblue', 'white', 'orangered', 'purple', 'darkgreen',  'silver']
#          0          1       2        3      4      5            6         7            8       9         10   11

ax.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels), cmap=matplotlib.colors.ListedColormap(colors), s=0.01)
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis('tight')

#ax = subplots[2, 1]
#ax.set_title("Perplexity=%d" % perplexity)
#colors = ['gray']
#ax.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_importance), cmap=matplotlib.colors.ListedColormap(colors), s=0.03)
#ax.xaxis.set_major_formatter(NullFormatter())
#ax.yaxis.set_major_formatter(NullFormatter())
#ax.axis('tight')


In [ ]:
fig.savefig('couzin-tsneplots', pad_inches=0)

In [ ]:
#TSNE single labels only

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

colors = ['blue', 'green', 'cyan', 'red', 'black']


fig = plt.figure(figsize=(10,10))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_importance), cmap=matplotlib.colors.ListedColormap(colors), s=0.03)
blue_patch = mpatches.Patch(color='blue', label='zor')
green_patch = mpatches.Patch(color='green', label='zoo')
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
red_patch = mpatches.Patch(color='red', label='avoid wall')
black_patch = mpatches.Patch(color='black', label='normal walk')
plt.legend(handles=[red_patch, blue_patch, green_patch, cyan_patch, black_patch])


In [ ]:
#TSNE single labels only

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

colors = ['blue', 'green', 'cyan', 'red', 'black']


fig = plt.figure(figsize=(10,10))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_importance), cmap=matplotlib.colors.ListedColormap(colors), s=0.03)
blue_patch = mpatches.Patch(color='blue', label='zor')
green_patch = mpatches.Patch(color='green', label='zoo')
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
red_patch = mpatches.Patch(color='red', label='avoid wall')
black_patch = mpatches.Patch(color='black', label='normal walk')
plt.legend(handles=[red_patch, blue_patch, green_patch, cyan_patch, black_patch])
#cb = plt.colorbar()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

colors = ['gray', 'red']


fig = plt.figure(figsize=(8,8))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_wall), cmap=matplotlib.colors.ListedColormap(colors), s=0.03)
red_patch = mpatches.Patch(color='red', label='avoid wall')
plt.legend(handles=[red_patch])
#cb = plt.colorbar()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

colors = ['gray', 'cyan']


fig = plt.figure(figsize=(8,8))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_zoa), cmap=matplotlib.colors.ListedColormap(colors), s=0.03)
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
plt.legend(handles=[cyan_patch])


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

colors = ['gray', 'green']


fig = plt.figure(figsize=(8,8))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_zoo), cmap=matplotlib.colors.ListedColormap(colors), s=0.03)
green_patch = mpatches.Patch(color='green', label='zoo')
plt.legend(handles=[green_patch])
#cb = plt.colorbar()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

colors = ['gray', 'blue']


fig = plt.figure(figsize=(8,8))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_zor), cmap=matplotlib.colors.ListedColormap(colors), s=0.03)
blue_patch = mpatches.Patch(color='blue', label='zor')
plt.legend(handles=[blue_patch])


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

colors = ['gray', 'black']


fig = plt.figure(figsize=(8,8))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels_normal), cmap=matplotlib.colors.ListedColormap(colors), s=0.3)
black_patch = mpatches.Patch(color='black', label='normal walk')
plt.legend(handles=[black_patch])


# UMAP

In [ ]:
import tables 
import numpy as np
import torch
from IPython.display import clear_output

hdf5_file = tables.open_file('pathtofile.h5', mode='r')
# features; 0: age; 1 bee_id; rest: normal data
train = hdf5_file.root.train
print(train.shape)
hidds = []
labels = []
labels_importance = []
labels_wall = []
labels_zoa = []
labels_zoo = []
labels_zor = []
printcounter = 0
for i in range(0, 10000):
    if (printcounter == 1000):
        clear_output()
        print('Progress report: ', i/100000)
        printcounter = 0
    printcounter += 1
    batch_data = train[i, :, :]
    label = batch_data[-1, :4]
    #print(label)
    #print(label[3])
    p = 1 
    if p >= np.random.uniform(0.0, 1.0):
        batch_X = batch_data[:-1, 10:].astype(np.float32)[None, :, :]
        batch_X = np.insert(batch_X,[1],batch_X[0],axis=0)
        batch_X = torch.from_numpy(batch_X)
        batch_X = torch.autograd.Variable(batch_X).cuda()
        Y_predicted, hidden = rnn.forward(batch_X, verbose=False)
        hidds.append(np.concatenate((hidden[0].data.cpu().numpy()[1, 0, :],hidden[1].data.cpu().numpy()[1, 0, :])))
        if np.where(label == 1)[0].shape[0] > 1:
            labels.append(np.where(label == 1)[0][1] + 1)
        elif np.where(label == 1)[0].shape[0] > 0:
            labels.append(np.where(label == 1)[0][0] + 1)
        else:
            labels.append(0)
        #print(label)
        if label[3] > 0:
            labels_importance.append(3) #avoid wall
        elif label[2] > 0:
            labels_importance.append(2) #attraction zone filled
        elif label[1] > 0:
            labels_importance.append(1) #orientation zone
        elif label[0]> 0:
            labels_importance.append(0) #repulsion zone
        else:
            labels_importance.append(4) # no label -> normal walk
            
        #------------------------------------------------
        labels_wall.append(label[3])
        labels_zoa.append(label[2])
        labels_zoo.append(label[1])
        labels_zor.append(label[0])
hdf5_file.close()

In [ ]:
data = np.asarray(hidds)
labels = np.asarray(labels)
labels_importance = np.asarray(labels_importance)
labels_wall = np.asarray(labels_wall)
labels_zoa = np.asarray(labels_zoa)
labels_zoo = np.asarray(labels_zoo)
labels_zor = np.asarray(labels_zor)
print(data.shape)
print(labels.shape)

In [ ]:
import umap

reducer = umap.UMAP(n_neighbors=100, min_dist=0.5)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
embedding = reducer.fit_transform(data)
embedding.shape

In [ ]:
colors = ['blue', 'green', 'cyan', 'red', 'black']


fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(embedding[:, 0], embedding[:, 1], c=labels_importance, cmap=matplotlib.colors.ListedColormap(colors), s=2)
plt.gca().set_aspect('equal', 'datalim')
blue_patch = mpatches.Patch(color='blue', label='zor')
green_patch = mpatches.Patch(color='green', label='zoo')
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
red_patch = mpatches.Patch(color='red', label='avoid wall')
black_patch = mpatches.Patch(color='black', label='normal walk')
plt.legend(handles=[red_patch, blue_patch, green_patch, cyan_patch, black_patch])
plt.colorbar(boundaries=np.arange(6)-0.5).set_ticks(np.arange(5))


plt.title('UMAP projection with normal Labels', fontsize=24);

# UMAP WITH LABELS

In [ ]:
import tables 
import numpy as np
import torch
from IPython.display import clear_output

hdf5_file = tables.open_file('pathtofile.h5', mode='r')
# features; 0: age; 1 bee_id; rest: normal data
train = hdf5_file.root.train
print(train.shape)
hidds_test = []
labels = []
labels_importance_test = []
labels_wall_test = []
labels_zoa_test = []
labels_zoo_test = []
labels_zor_test = []
printcounter = 0
for i in range(0, 10000):
    if (printcounter == 1000):
        clear_output()
        print('Progress report: ', i/100000)
        printcounter = 0
    printcounter += 1
    batch_data = train[i, :, :]
    label = batch_data[-1, :4]
    #print(label)
    #print(label[3])
    p = 1 
    if p >= np.random.uniform(0.0, 1.0):
        batch_X = batch_data[:-1, 10:].astype(np.float32)[None, :, :]
        batch_X = np.insert(batch_X,[1],batch_X[0],axis=0)
        batch_X = torch.from_numpy(batch_X)
        batch_X = torch.autograd.Variable(batch_X).cuda()
        Y_predicted, hidden = rnn.forward(batch_X, verbose=False)
        hidds_test.append(np.concatenate((hidden[0].data.cpu().numpy()[1, 0, :],hidden[1].data.cpu().numpy()[1, 0, :])))
        if np.where(label == 1)[0].shape[0] > 1:
            labels.append(np.where(label == 1)[0][1] + 1)
        elif np.where(label == 1)[0].shape[0] > 0:
            labels.append(np.where(label == 1)[0][0] + 1)
        else:
            labels.append(0)
        #print(label)
        if label[3] > 0:
            labels_importance_test.append(3) #avoid wall
        elif label[2] > 0:
            labels_importance_test.append(2) #attraction zone filled
        elif label[1] > 0:
            labels_importance_test.append(1) #orientation zone
        elif label[0]> 0:
            labels_importance_test.append(0) #repulsion zone
        else:
            labels_importance_test.append(4) # no label -> normal walk
            
        #------------------------------------------------
        labels_wall_test.append(label[3])
        labels_zoa_test.append(label[2])
        labels_zoo_test.append(label[1])
        labels_zor_test.append(label[0])
hdf5_file.close()

In [ ]:
test_data = np.asarray(hidds_test)
labels = np.asarray(labels)
labels_importance_test = np.asarray(labels_importance_test)
labels_wall_test = np.asarray(labels_wall_test)
labels_zoa_test = np.asarray(labels_zoa_test)
labels_zoo_test = np.asarray(labels_zoo_test)
labels_zor_test = np.asarray(labels_zor_test)
print(test_data.shape)
print(labels_importance_test.shape)

In [ ]:
#mapper = umap.UMAP(min_dist=1, n_neighbors=10).fit(data, y=labels_neighs_digitzized_test)
mapper = umap.UMAP(n_neighbors=89, min_dist=0.5).fit(data, y=labels_importance)

In [ ]:
test_embedding = mapper.transform(test_data)

In [ ]:
colors = ['blue', 'green', 'cyan', 'red', 'black']

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*mapper.embedding_.T, s=0.3, c=labels_importance, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
red_patch = mpatches.Patch(color='red', label='avoid wall')
blue_patch = mpatches.Patch(color='blue', label='zor')
green_patch = mpatches.Patch(color='green', label='zoo')
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
plt.legend(handles=[red_patch, blue_patch, green_patch, cyan_patch])
plt.title('Train Data Embedded via UMAP');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*test_embedding.T, s=0.3, c=labels_importance_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
red_patch = mpatches.Patch(color='red', label='avoid wall')
blue_patch = mpatches.Patch(color='blue', label='zor')
green_patch = mpatches.Patch(color='green', label='zoo')
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
plt.legend(handles=[red_patch, blue_patch, green_patch, cyan_patch])
plt.title('Data Embedded via UMAP');

In [ ]:
from mpl_toolkits.mplot3d import axes3d, Axes3D 

fit = umap.UMAP(
n_neighbors=89,
min_dist= 0.7,
n_components= 3
)
u = fit.fit_transform(data, y=labels_importance);

In [ ]:
tu = fit.transform(test_data)

In [ ]:
colors = ['blue', 'white', 'cyan', 'red', 'black']

In [ ]:
fig = plt.figure(figsize=(16, 12))
ax = Axes3D(fig)
#ax = fig.add_subplot(111, projection='3d')
ax.scatter(u[:,0], u[:,1], u[:,2], c=labels_importance, cmap=matplotlib.colors.ListedColormap(colors), s=3, alpha = 1.0)
plt.title("title", fontsize=18)
#ax.view_init(0, 0)


In [ ]:
fig = plt.figure(figsize=(16, 12))
ax = Axes3D(fig)
#ax = fig.add_subplot(111, projection='3d')
ax.scatter(tu[:,0], tu[:,1], tu[:,2], c=labels_importance_test, cmap=matplotlib.colors.ListedColormap(colors), s=3, alpha = 1.0)
plt.title("title", fontsize=18)
#ax.view_init(0, 0)


In [ ]:
fig = plt.figure(figsize=(16, 12))
ax = Axes3D(fig)
#ax = fig.add_subplot(111, projection='3d')
ax.scatter(u[:,0], u[:,1], u[:,2], c=labels_importance, cmap=matplotlib.colors.ListedColormap(colors), s=3, alpha = 1.0)
plt.title("title", fontsize=18)
ax.view_init(0, 20)

    

In [ ]:
fig = plt.figure(figsize=(16, 12))
ax = Axes3D(fig)
#ax = fig.add_subplot(111, projection='3d')
ax.scatter(tu[:,0], tu[:,1], tu[:,2], c=labels_importance_test, cmap=matplotlib.colors.ListedColormap(colors), s=3, alpha = 1.0)
plt.title("title", fontsize=18)
ax.view_init(0, 20)

    

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*mapper.embedding_.T, s=0.3, c=labels_importance, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
red_patch = mpatches.Patch(color='red', label='avoid wall')
blue_patch = mpatches.Patch(color='blue', label='zor')
green_patch = mpatches.Patch(color='green', label='zoo')
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
plt.legend(handles=[red_patch, blue_patch, green_patch, cyan_patch])
plt.title('Train Data Embedded via UMAP');

In [ ]:
colors = ['gray', 'red']

In [ ]:
#mapper = umap.UMAP(min_dist=1, n_neighbors=10).fit(data, y=labels_neighs_digitzized_test)
mapper = umap.UMAP(n_neighbors=89, min_dist=0.5).fit(data, y=labels_wall)

In [ ]:
test_embedding = mapper.transform(test_data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*mapper.embedding_.T, s=0.2, c=labels_wall, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
red_patch = mpatches.Patch(color='red', label='avoid wall')
plt.legend(handles=[red_patch])
plt.title('Train Data Embedded via UMAP');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*test_embedding.T, s=0.2, c=labels_wall_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
red_patch = mpatches.Patch(color='red', label='avoid wall')
plt.legend(handles=[red_patch])
plt.title('Data Embedded via UMAP');

In [ ]:
colors = ['gray', 'blue']

In [ ]:
#mapper = umap.UMAP(min_dist=1, n_neighbors=10).fit(data, y=labels_neighs_digitzized_test)
mapper = umap.UMAP(n_neighbors=89, min_dist=0.5).fit(data, y=labels_zor)

In [ ]:
test_embedding = mapper.transform(test_data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*mapper.embedding_.T, s=0.2, c=labels_zor, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
blue_patch = mpatches.Patch(color='blue', label='zor')
plt.legend(handles=[blue_patch])
plt.title('Train Data Embedded via UMAP');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*test_embedding.T, s=0.2, c=labels_zor_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
blue_patch = mpatches.Patch(color='blue', label='zor')
plt.legend(handles=[blue_patch])
plt.title('Data Embedded via UMAP');

In [ ]:
colors = ['gray', 'green']

In [ ]:
#mapper = umap.UMAP(min_dist=1, n_neighbors=10).fit(data, y=labels_neighs_digitzized_test)
mapper = umap.UMAP(n_neighbors=89, min_dist=0.5).fit(data, y=labels_zoo)

In [ ]:
test_embedding = mapper.transform(test_data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*mapper.embedding_.T, s=0.2, c=labels_zoo, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
green_patch = mpatches.Patch(color='green', label='zoo')
plt.legend(handles=[green_patch])
plt.title('Train Data Embedded via UMAP');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*test_embedding.T, s=0.2, c=labels_zoo_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
green_patch = mpatches.Patch(color='green', label='zoo')
plt.legend(handles=[green_patch])
plt.title('Data Embedded via UMAP');

In [ ]:
colors = ['gray', 'cyan']

In [ ]:
#mapper = umap.UMAP(min_dist=1, n_neighbors=10).fit(data, y=labels_neighs_digitzized_test)
mapper = umap.UMAP(n_neighbors=89, min_dist=0.5).fit(data, y=labels_zoa)

In [ ]:
test_embedding = mapper.transform(test_data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*mapper.embedding_.T, s=0.2, c=labels_zoa, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
plt.legend(handles=[cyan_patch])
plt.title('Train Data Embedded via UMAP');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*test_embedding.T, s=0.2, c=labels_zoa_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
plt.legend(handles=[cyan_patch])
plt.title('Data Embedded via UMAP');

In [ ]:
import tables 
import numpy as np
import torch
from IPython.display import clear_output

hdf5_file = tables.open_file('pathtofile.h5', mode='r')
# features; 0: age; 1 bee_id; rest: normal data
train = hdf5_file.root.train
print(train.shape)
hidds = []
hidds_only = []
labels = []
labels_importance = []
labels_wall = []
labels_zoa = []
labels_zoo = []
labels_zor = []
labels_wall_only = []
labels_zoa_only = []
labels_zoo_only = []
labels_zor_only = []
labels_0__only = []
labels_importance_only = []
printcounter = 0
for i in range(0, 100000):
    if (printcounter == 1000):
        clear_output()
        print('Progress report: ', i/100000)
        printcounter = 0
    printcounter += 1
    batch_data = train[i, :, :]
    label = batch_data[-1, :4]
    #print(label)
    #print(label[3])
    p = 1 
    if p >= np.random.uniform(0.0, 1.0):
        batch_X = batch_data[:-1, 10:].astype(np.float32)[None, :, :]
        batch_X = np.insert(batch_X,[1],batch_X[0],axis=0)
        batch_X = torch.from_numpy(batch_X)
        batch_X = torch.autograd.Variable(batch_X).cuda()
        Y_predicted, hidden = rnn.forward(batch_X, verbose=False)
        hidds.append(np.concatenate((hidden[0].data.cpu().numpy()[1, 0, :],hidden[1].data.cpu().numpy()[1, 0, :])))
        if np.where(label == 1)[0].shape[0] > 1:
            labels.append(np.where(label == 1)[0][1] + 1)
        elif np.where(label == 1)[0].shape[0] > 0:
            labels.append(np.where(label == 1)[0][0] + 1)
        else:
            labels.append(0)
        #print(label)
        if label[3] > 0:
            labels_importance.append(3) #avoid wall
        elif label[2] > 0:
            labels_importance.append(2) #attraction zone filled
        elif label[1] > 0:
            labels_importance.append(1) #orientation zone
        elif label[0]> 0:
            labels_importance.append(0) #repulsion zone
        else:
            labels_importance.append(4) # no label -> normal walk
            
        #------------------------------------------------
        labels_wall.append(label[3])
        labels_zoa.append(label[2])
        labels_zoo.append(label[1])
        labels_zor.append(label[0])
        
        if np.sum(label) <= 1:
            hidds_only.append(np.concatenate((hidden[0].data.cpu().numpy()[1, 0, :],hidden[1].data.cpu().numpy()[1, 0, :])))
            if np.array_equal(np.array(label), np.array([0,0,0,0])):
                labels_0__only.append(1)
            else:
                labels_0__only.append(0)
            labels_wall_only.append(label[3])
            labels_zoa_only.append(label[2])
            labels_zoo_only.append(label[1])
            labels_zor_only.append(label[0])
            
            if label[3] > 0:
                labels_importance_only.append(3) #avoid wall
            elif label[2] > 0:
                labels_importance_only.append(2) #attraction zone filled
            elif label[1] > 0:
                labels_importance_only.append(1) #orientation zone
            elif label[0]> 0:
                labels_importance_only.append(0) #repulsion zone
            else:
                labels_importance_only.append(4) # no label -> normal walk
hdf5_file.close()

In [ ]:
data = np.asarray(hidds)
labels = np.asarray(labels)
labels_importance = np.asarray(labels_importance)
labels_wall = np.asarray(labels_wall)
labels_zoa = np.asarray(labels_zoa)
labels_zoo = np.asarray(labels_zoo)
labels_zor = np.asarray(labels_zor)

data_only = np.asarray(hidds_only)
labels_wall_only = np.asarray(labels_wall_only)
labels_zoa_only = np.asarray(labels_zoa_only)
labels_zoo_only = np.asarray(labels_zoo_only)
labels_zor_only = np.asarray(labels_zor_only)
labels_importance_only = np.asarray(labels_importance_only)
print(data.shape)
print(labels.shape)

In [ ]:
print("labels_wall: ", np.sum(labels_wall_only))
print("labels_zoa: ", np.sum(labels_zoa_only))
print("labels_zoo: ", np.sum(labels_zoo_only))
print("labels_zor: ", np.sum(labels_zor_only))
print("labels_importance_only: ", np.sum(labels_importance_only))

In [ ]:
import tables 
import numpy as np
import torch
from IPython.display import clear_output

hdf5_file = tables.open_file('pathtofile.h5', mode='r')
# features; 0: age; 1 bee_id; rest: normal data
train = hdf5_file.root.train
print(train.shape)
hidds_test = []
hidds_only_test = []
labels_test = []
labels_importance_test = []
labels_wall_test = []
labels_zoa_test = []
labels_zoo_test = []
labels_zor_test = []
labels_wall_only_test = []
labels_zoa_only_test = []
labels_zoo_only_test = []
labels_zor_only_test = []
labels_0__only_test = []
labels_importance_only_test = []
printcounter = 0
for i in range(0, 100000):
    if (printcounter == 1000):
        clear_output()
        print('Progress report: ', i/100000)
        printcounter = 0
    printcounter += 1
    batch_data = train[i, :, :]
    label = batch_data[-1, :4]
    #print(label)
    #print(label[3])
    p = 1 
    if p >= np.random.uniform(0.0, 1.0):
        batch_X = batch_data[:-1, 10:].astype(np.float32)[None, :, :]
        batch_X = np.insert(batch_X,[1],batch_X[0],axis=0)
        batch_X = torch.from_numpy(batch_X)
        batch_X = torch.autograd.Variable(batch_X).cuda()
        Y_predicted, hidden = rnn.forward(batch_X, verbose=False)
        hidds_test.append(np.concatenate((hidden[0].data.cpu().numpy()[1, 0, :],hidden[1].data.cpu().numpy()[1, 0, :])))
        if np.where(label == 1)[0].shape[0] > 1:
            labels_test.append(np.where(label == 1)[0][1] + 1)
        elif np.where(label == 1)[0].shape[0] > 0:
            labels_test.append(np.where(label == 1)[0][0] + 1)
        else:
            labels_test.append(0)
        #print(label)
        if label[3] > 0:
            labels_importance_test.append(3) #avoid wall
        elif label[2] > 0:
            labels_importance_test.append(2) #attraction zone filled
        elif label[1] > 0:
            labels_importance_test.append(1) #orientation zone
        elif label[0]> 0:
            labels_importance_test.append(0) #repulsion zone
        else:
            labels_importance_test.append(4) # no label -> normal walk
            
        #------------------------------------------------
        labels_wall_test.append(label[3])
        labels_zoa_test.append(label[2])
        labels_zoo_test.append(label[1])
        labels_zor_test.append(label[0])
        
        if np.sum(label) <= 1:
            hidds_only_test.append(np.concatenate((hidden[0].data.cpu().numpy()[1, 0, :],hidden[1].data.cpu().numpy()[1, 0, :])))
            if np.array_equal(np.array(label), np.array([0,0,0,0])):
                labels_0__only_test.append(1)
            else:
                labels_0__only_test.append(0)
            labels_wall_only_test.append(label[3])
            labels_zoa_only_test.append(label[2])
            labels_zoo_only_test.append(label[1])
            labels_zor_only_test.append(label[0])
            
            if label[3] > 0:
                labels_importance_only_test.append(3) #avoid wall
            elif label[2] > 0:
                labels_importance_only_test.append(2) #attraction zone filled
            elif label[1] > 0:
                labels_importance_only_test.append(1) #orientation zone
            elif label[0]> 0:
                labels_importance_only_test.append(0) #repulsion zone
            else:
                labels_importance_only_test.append(4) # no label -> normal walk
hdf5_file.close()

In [ ]:
test_data = np.asarray(hidds_test)
labels_test = np.asarray(labels)
labels_importance_test = np.asarray(labels_importance_test)
labels_wall_test = np.asarray(labels_wall_test)
labels_zoa_test = np.asarray(labels_zoa_test)
labels_zoo_test = np.asarray(labels_zoo_test)
labels_zor_test = np.asarray(labels_zor_test)

test_data_only = np.asarray(hidds_only_test)
labels_importance_only_test = np.asarray(labels_importance_only_test)
labels_wall_only_test = np.asarray(labels_wall_only_test)
labels_zoa_only_test = np.asarray(labels_zoa_only_test)
labels_zoo_only_test = np.asarray(labels_zoo_only_test)
labels_zor_only_test = np.asarray(labels_zor_only_test)
print(test_data.shape)
print(labels_importance_test.shape)

In [ ]:
#mapper = umap.UMAP(min_dist=1, n_neighbors=10).fit(data, y=labels_neighs_digitzized_test)
mapper = umap.UMAP(n_neighbors=89, min_dist=0.5).fit(data_only, y=labels_importance_only)

In [ ]:
test_embedding = mapper.transform(test_data_only)

In [ ]:
colors = ['blue', 'green', 'cyan', 'red', 'black']

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*mapper.embedding_.T, s=0.05, c=labels_importance_only, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
red_patch = mpatches.Patch(color='red', label='avoid wall')
blue_patch = mpatches.Patch(color='blue', label='zor')
green_patch = mpatches.Patch(color='green', label='zoo')
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
plt.legend(handles=[red_patch, blue_patch, green_patch, cyan_patch])
plt.title('Train Data Embedded via UMAP');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*test_embedding.T, s=0.3, c=labels_importance_only_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
red_patch = mpatches.Patch(color='red', label='avoid wall')
blue_patch = mpatches.Patch(color='blue', label='zor')
green_patch = mpatches.Patch(color='green', label='zoo')
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
plt.legend(handles=[red_patch, blue_patch, green_patch, cyan_patch])
plt.title('Data Embedded via UMAP');

In [ ]:
#mapper = umap.UMAP(min_dist=1, n_neighbors=10).fit(data, y=labels_neighs_digitzized_test)
mapper = umap.UMAP(n_neighbors=89, min_dist=0.5).fit(data_only, y=labels_wall_only)

In [ ]:
test_embedding = mapper.transform(test_data_only)

In [ ]:
colors = ['gray', 'red']

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*mapper.embedding_.T, s=2, c=labels_wall_only, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
red_patch = mpatches.Patch(color='red', label='avoid wall')
plt.legend(handles=[red_patch])
plt.title('Train Data Embedded via UMAP');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*test_embedding.T, s=0.2, c=labels_wall_only_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
red_patch = mpatches.Patch(color='red', label='avoid wall')
plt.legend(handles=[red_patch])
plt.title('Data Embedded via UMAP');

In [ ]:
colors = ['gray', 'blue']

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*mapper.embedding_.T, s=0.2, c=labels_zor_only, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
blue_patch = mpatches.Patch(color='blue', label='zor')
plt.legend(handles=[blue_patch])
plt.title('Train Data Embedded via UMAP');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*test_embedding.T, s=0.2, c=labels_zor_only_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
blue_patch = mpatches.Patch(color='blue', label='zor')
plt.legend(handles=[blue_patch])
plt.title('Data Embedded via UMAP');

In [ ]:
colors = ['gray', 'green']

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*mapper.embedding_.T, s=0.2, c=labels_zoo_only, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
green_patch = mpatches.Patch(color='green', label='zoo')
plt.legend(handles=[green_patch])
plt.title('Train Data Embedded via UMAP');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*test_embedding.T, s=0.2, c=labels_zoo_only_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
green_patch = mpatches.Patch(color='green', label='zoo')
plt.legend(handles=[green_patch])
plt.title('Data Embedded via UMAP');

In [ ]:
colors = ['gray', 'cyan']

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*mapper.embedding_.T, s=0.2, c=labels_zoa_only, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
plt.legend(handles=[cyan_patch])
plt.title('Train Data Embedded via UMAP');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*test_embedding.T, s=0.2, c=labels_zoa_only_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
cyan_patch = mpatches.Patch(color='cyan', label='zoa')
plt.legend(handles=[cyan_patch])
plt.title('Data Embedded via UMAP');

In [ ]:
import tables 
import numpy as np
import torch
from IPython.display import clear_output

hdf5_file = tables.open_file('pathtofile.h5', mode='r')
# features; 0: age; 1 bee_id; rest: normal data
train = hdf5_file.root.train
print(train.shape)
hidds = []
labels = []
printcounter = 0
for i in range(0, 100000):
    if (printcounter == 1000):
        clear_output()
        print('Progress report: ', i/100000)
        printcounter = 0
    printcounter += 1
    batch_data = train[i, :, :]
    label = batch_data[-1, :4]
    #print(label)
    #print(label[3])
    p = 1 
    if p >= np.random.uniform(0.0, 1.0):
        batch_X = batch_data[:-1, 10:].astype(np.float32)[None, :, :]
        batch_X = np.insert(batch_X,[1],batch_X[0],axis=0)
        batch_X = torch.from_numpy(batch_X)
        batch_X = torch.autograd.Variable(batch_X).cuda()
        Y_predicted, hidden = rnn.forward(batch_X, verbose=False)
        hidds.append(np.concatenate((hidden[0].data.cpu().numpy()[1, 0, :],hidden[1].data.cpu().numpy()[1, 0, :])))

        if np.array_equal(label, np.array([0,0,0,0])):
            labels.append(0)
        elif np.array_equal(label, np.array([1,0,0,0])):
            labels.append(1)
        elif np.array_equal(label, np.array([0,1,0,0])):
            labels.append(2)
        elif np.array_equal(label, np.array([0,0,1,0])):
            labels.append(3)
        elif np.array_equal(label, np.array([0,0,0,1])):
            labels.append(4)
        elif np.array_equal(label, np.array([1,1,0,0])):
            labels.append(5)
        elif np.array_equal(label, np.array([1,0,1,0])):
            labels.append(6)
        elif np.array_equal(label, np.array([1,0,0,1])):
            labels.append(7)
        elif np.array_equal(label, np.array([0,1,1,0])):
            labels.append(8)
        elif np.array_equal(label, np.array([0,1,0,1])):
            labels.append(9)
        elif np.array_equal(label, np.array([0,0,1,1])):
            labels.append(10) 
        elif np.array_equal(label, np.array([1,1,1,0])):
            labels.append(11)
        elif np.array_equal(label, np.array([1,1,0,1])):
            labels.append(12)
        elif np.array_equal(label, np.array([1,0,1,1])):
            labels.append(13)
        elif np.array_equal(label, np.array([0,1,1,1])):
            labels.append(14)
        elif np.array_equal(label, np.array([1,1,1,1])):
            labels.append(15)
        else:
            print("NOOOO")

hdf5_file.close()

In [ ]:
plt.hist(labels, range(0,17))
plt.yscale('log')


In [ ]:
colors = ['black', 'red', 'yellow', 'blue', 'gray', 'orange', 'pink', 'darkred', 'green', 'khaki', 'darkblue', 'ivory', 'orangered', 'purple', 'darkgreen',  'silver']


In [ ]:
x = np.arange(16)
money = np.histogram(labels, bins= np.arange(17))[0]


#formatter = FuncFormatter(millions)

fig, ax = plt.subplots(figsize=(12, 12), dpi=95)
#ax.yaxis.set_major_formatter(formatter)
plt.bar(x, money, color=colors)
plt.yscale('log')
plt.xticks(x, ('normal', 'zor', 'zoo', 'zoa', 'wall', '1100', '1010', '1001', '0110', '0101', '0011',
               '1110', '1101', '1011', '0111', '1111'))
plt.xlabel('Label combination of window')
plt.ylabel('Number of windows')
plt.show()

In [ ]:
hidds = np.asarray(hidds)
labels = np.asarray(labels)
print(hidds.shape)
print(labels.shape)

In [ ]:
#from sklearn.manifold import TSNE
#https://github.com/DmitryUlyanov/Multicore-TSNE}
from MulticoreTSNE import MulticoreTSNE as TSNE

X_embedded = TSNE(n_components=2, perplexity = 50).fit_transform(hidds)
X_embedded.shape

In [ ]:
#TSNE single labels only

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

colors = ['black', 'red', 'yellow', 'blue', 'gray', 'orange', 'pink', 'darkred', 'green', 'khaki', 'darkblue', 'white', 'orangered', 'purple', 'darkgreen',  'silver']
#          0          1       2        3      4      5            6         7            8       9         10   11

fig = plt.figure(figsize=(20,20))
#plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels), cmap=matplotlib.colors.ListedColormap(colors), s=0.3)
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels), cmap=matplotlib.colors.ListedColormap(colors), s=5)
#cb = plt.colorbar()

In [ ]:
#from sklearn.manifold import TSNE
#https://github.com/DmitryUlyanov/Multicore-TSNE}
from MulticoreTSNE import MulticoreTSNE as TSNE

X_embedded = TSNE(n_components=2, perplexity = 97).fit_transform(hidds)
X_embedded.shape

In [ ]:
#TSNE single labels only

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

colors = ['black', 'red', 'yellow', 'blue', 'gray', 'orange', 'pink', 'darkred', 'green', 'khaki', 'darkblue', 'white', 'orangered', 'purple', 'darkgreen',  'silver']
#          0          1       2        3      4      5            6         7            8       9         10   11

fig = plt.figure(figsize=(20,20))
#plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels), cmap=matplotlib.colors.ListedColormap(colors), s=0.3)
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=np.round(labels), cmap=matplotlib.colors.ListedColormap(colors), s=0.75)
#cb = plt.colorbar()

In [ ]:
import tables 
import numpy as np
import torch
from IPython.display import clear_output

hdf5_file = tables.open_file('/media/mehmed/ssd1tb/bb-predict-data/couzin_tsneumap_test.h5', mode='r')
# features; 0: age; 1 bee_id; rest: normal data
train = hdf5_file.root.train
print(train.shape)
hidds_test = []
labels_test = []
printcounter = 0
for i in range(0, 100000):
    if (printcounter == 1000):
        clear_output()
        print('Progress report: ', i/100000)
        printcounter = 0
    printcounter += 1
    batch_data = train[i, :, :]
    label = batch_data[-1, :4]
    #print(label)
    #print(label[3])
    p = 1 
    if p >= np.random.uniform(0.0, 1.0):
        batch_X = batch_data[:-1, 10:].astype(np.float32)[None, :, :]
        batch_X = np.insert(batch_X,[1],batch_X[0],axis=0)
        batch_X = torch.from_numpy(batch_X)
        batch_X = torch.autograd.Variable(batch_X).cuda()
        Y_predicted, hidden = rnn.forward(batch_X, verbose=False)
        hidds_test.append(np.concatenate((hidden[0].data.cpu().numpy()[1, 0, :],hidden[1].data.cpu().numpy()[1, 0, :])))

        if np.array_equal(label, np.array([0,0,0,0])):
            labels_test.append(0)
        elif np.array_equal(label, np.array([1,0,0,0])):
            labels_test.append(1)
        elif np.array_equal(label, np.array([0,1,0,0])):
            labels_test.append(2)
        elif np.array_equal(label, np.array([0,0,1,0])):
            labels_test.append(3)
        elif np.array_equal(label, np.array([0,0,0,1])):
            labels_test.append(4)
        elif np.array_equal(label, np.array([1,1,0,0])):
            labels_test.append(5)
        elif np.array_equal(label, np.array([1,0,1,0])):
            labels_test.append(6)
        elif np.array_equal(label, np.array([1,0,0,1])):
            labels_test.append(7)
        elif np.array_equal(label, np.array([0,1,1,0])):
            labels_test.append(8)
        elif np.array_equal(label, np.array([0,1,0,1])):
            labels_test.append(9)
        elif np.array_equal(label, np.array([0,0,1,1])):
            labels_test.append(10) 
        elif np.array_equal(label, np.array([1,1,1,0])):
            labels_test.append(11)
        elif np.array_equal(label, np.array([1,1,0,1])):
            labels_test.append(12)
        elif np.array_equal(label, np.array([1,0,1,1])):
            labels_test.append(13)
        elif np.array_equal(label, np.array([0,1,1,1])):
            labels_test.append(14)
        elif np.array_equal(label, np.array([1,1,1,1])):
            labels_test.append(15)
        else:
            print("NOOOO") 
hdf5_file.close()

In [ ]:
test_data = np.asarray(hidds_test)
labels_test = np.asarray(labels_test)

print(test_data.shape)
print(labels_test.shape)

In [ ]:
import umap

In [ ]:
#mapper = umap.UMAP(min_dist=1, n_neighbors=10).fit(data, y=labels_neighs_digitzized_test)
mapper = umap.UMAP(n_neighbors=89, min_dist=0.5).fit(hidds, y=labels)

In [ ]:
test_embedding = mapper.transform(test_data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))
plt.scatter(*mapper.embedding_.T, s=0.5, c=labels, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
#cb = plt.colorbar()
plt.title('Train data embedded via UMAP');

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))
plt.scatter(*test_embedding.T, s=0.5, c=labels_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
#cb = plt.colorbar()
plt.title('Test data embedded via UMAP');

In [ ]:
from matplotlib.ticker import NullFormatter

perplexities = [0, 5, 30, 50, 100]
(fig, subplots) = plt.subplots(1, 2, figsize=(20, 10))

fig.subplots_adjust(wspace=0.02, hspace=0.02)

ax = subplots[0]
#ax.set_title("Perplexity=%d" % perplexity)
ax.scatter(*mapper.embedding_.T, s=0.5, c=labels, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
#cb = plt.colorbar()
ax.set_title('Train data embedded via UMAP')
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis('tight')

ax = subplots[1]
ax.scatter(*test_embedding.T, s=0.5, c=labels_test, cmap=matplotlib.colors.ListedColormap(colors), alpha=1.0)
ax.set_title('Test data embedded via UMAP')
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis('tight')

